# Installations, Imports and Downloads


In [1]:
import os
import warnings
import re
import numpy as np
warnings.filterwarnings("ignore", category = UserWarning)
from collections import defaultdict
import pandas as pd
import jax.numpy as jnp
from jax import grad
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

FS = (8,4) # figure size
RS = 264


In [2]:
PROJECT_DIR = os.getcwd() + '/afrisent-semeval-2023'
print('Current directiory: ', PROJECT_DIR)
PROJECT_GITHUB_URL = 'https://github.com/afrisenti-semeval/afrisent-semeval-2023.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  !git pull {PROJECT_GITHUB_URL}

Current directiory:  /Users/rogerbukuru/Documents/UCT Masters/MSc Statistics and Data Science/NLP-CSC5035Z/NLPTutsAssignments/Assignment-I/afrisent-semeval-2023
/Users/rogerbukuru/Documents/UCT Masters/MSc Statistics and Data Science/NLP-CSC5035Z/NLPTutsAssignments/Assignment-I/afrisent-semeval-2023
From https://github.com/afrisenti-semeval/afrisent-semeval-2023
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


# Data Loading


In [17]:
# Choose language 
language = 'swa'

In [18]:
# Load data
DATA_DIR = f'{PROJECT_DIR}/data/{language}'
print('Data directory: ', DATA_DIR)

train_df = pd.read_csv(f'{DATA_DIR}/train.tsv', sep='\t', names=['text', 'label'], header=0)
dev_df = pd.read_csv(f'{DATA_DIR}/dev.tsv', sep='\t', names=['text', 'label'], header=0)
test_df = pd.read_csv(f'{DATA_DIR}/test.tsv', sep='\t', names=['text', 'label'], header=0)

print('Train shape: ', train_df.shape)
print('Dev shape: ', dev_df.shape)
print('Test shape: ', test_df.shape)

# Display data
train_df.sample(n=10)

Data directory:  /Users/rogerbukuru/Documents/UCT Masters/MSc Statistics and Data Science/NLP-CSC5035Z/NLPTutsAssignments/Assignment-I/afrisent-semeval-2023/data/swa
Train shape:  (1810, 2)
Dev shape:  (453, 2)
Test shape:  (748, 2)


,text,label
100,Hili tatizo kubwa sana kwetumtu akijaribu kutu...,negative
559,Wameambiwa na mwenyekiti wao kila kitu ni sawa...,neutral
1440,Kumbe tunaifukuzia China kwa wingitushafika bi...,positive
119,Show nyingi za kibongo Show ikiisha Shabiki un...,negative
1473,Katika kuhakikisha tunafikia 5050 Wanahabari w...,positive
528,Hili jambo sio dogo kama tunavyofikiriahii eli...,neutral
1456,Hii ni Hatua Muhimu sana pongezi kwa kikao hic...,positive
138,Kama uamini kaamuulize Agger karacai msanii wa...,negative
605,Habari ahsante kwa kuwasiliana nasi Naomba ufa...,neutral
276,Ujumbe huo unamaanisha haukujitoa katika hudum...,neutral


# Data Cleaning

In [19]:
# Discard neutral examples
train_df = train_df[train_df['label'] != 'neutral']
dev_df = dev_df[dev_df['label'] != 'neutral']
test_df = test_df[test_df['label'] != 'neutral']

In [20]:
def clean(text):
    # Replace URLS with [URL]
    text = re.sub(r'http\S+', '[URL]', text)

    # Replace numbers with [NUM]
    text = re.sub(r'\d+', '[NUM]', text)

    # Remove trailing spaces
    text = text.strip()

    return text

train_df['text'] = train_df['text'].apply(clean)
dev_df['text'] = dev_df['text'].apply(clean)
test_df['text'] = test_df['text'].apply(clean)

# Construct Vocabulary

In [7]:
# Count number of tokens in corpus
def count_tokens(sentences):
    """
    Count number of tokens in corpus

    param: sentences: list of list of tokens e.g. [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence'], ...]
    return:
        count: number of tokens in corpus
    """
    total_tokens = 0
    for sentence in sentences:
        total_tokens += len(sentence)
    return total_tokens

In [8]:
# Collect type counts in corpus
def create_type_counts(sentences):
    """
    Count number of types in corpus

    param: sentences: list of list of tokens e.g. [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence'], ...]
    return:
        type2count: dictionary of type counts in corpus e.g. {'This': 2, 'sentence': 2, ...}
    """
    type2count = {}
    for sentence in sentences:
        for type_ in sentence:
            if type_ not in type2count:
                type2count[type_] = 1
            else:
                current_count = type2count[type_]
                type2count[type_] = current_count +1
    return type2count

In [9]:
# Create vocabulary
def create_vocabulary(type2count, min_count):
    """
    This function creates an indexed vocabulary from vocabulary counts and returns it as a list and a dictionary.

    param:
        type2count: dictionary of type counts in corpus e.g. {'This': 2, 'sentence': 2, ...}
        min_count: minimum count of a word to be included in the vocabulary
    return:
        index2type: list of words in the vocabulary e.g. ['word1', 'word2', 'word3', ...]
        type2index: dictionary mapping words to their index in the index2type vocabulary e.g. {'word1': 0, 'word2': 1, 'word3': 2, ...}
    """
    index2type = []
    type2index = {}
    for type_, count in type2count.items():
        if(count >= min_count):
            index2type.append(type_)
            type2index[type_] = len(index2type) - 1
    return index2type, type2index




# Tokenization

## Word Based Tokenization

In [10]:
def whitespace_tokenize(sentences):
    return [sentence.split() for sentence in sentences]

## Byte-Pair Encoding (BPE) Tokenization


In [11]:
class BPETokenizer():

    def __init__(self, sentences, vocab_size):
        """
        Initialize the BPE tokenizer.

        Args:
            sentences (list[str]): list of list of tokens e.g. [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence'], ...]
            vocab_size (int): The desired vocabulary size after training.
        """
        self.sentences = sentences
        self.vocab_size = vocab_size
        self.word_freqs = defaultdict(int)
        self.splits = {}
        self.merges = {}


    def train(self):
        """
        Train the BPE tokenizer by iteratively merging the most frequent pairs of symbols.

        Returns:
            dict: A dictionary of merges in the format {(a, b): 'ab'}, where 'a' and 'b' are symbols merged into 'ab'.
        """
        # Split corpus
        for sentence in self.sentences:
            for word in sentence:
                self.splits[word] = [char for char in word]
                    
        for i in range(self.vocab_size):
            self.compute_pair_freqs() # compute adjacent pair frequencies
            pair, _ = list(self.word_freqs.items())[0] # most frequent pair
            self.merge_pair(pair[0], pair[1])
            self.merges[pair] = pair[0] + pair[1]
        return self.merges


    def compute_pair_freqs(self):
        """
        Compute the frequency of each pair of symbols in the corpus.

        Returns:
            dict: A dictionary of pairs and their frequencies in the format {(a, b): frequency}.
        """
        pair_freqs = defaultdict(int)
        for _, split in self.splits.items():
            for i in range(len(split)-1):
                pair = (split[i], split[i+1])
                if pair not in pair_freqs:
                    pair_freqs[pair] = 1
                else:
                    pair_freqs[pair] += 1
        self.word_freqs = pair_freqs
        self.word_freqs = dict(sorted(self.word_freqs.items(), key=lambda x: x[1], reverse=True)) # sort from max to min count
        return self.word_freqs
        
    def merge_pair(self, a, b):
        """
        Merge the given pair of symbols in all words where they appear adjacent.

        Args:
            a (str): The first symbol in the pair.
            b (str): The second symbol in the pair.

        Returns:
            dict: The updated splits dictionary after merging.
        """
        pair = (a,b)
        # Check if valid pair
        if pair in self.word_freqs:
            new_token = a+b
            for word, split in self.splits.items():
                for i in range(len(split)-1):
                    if split[i] == a and split[i+1] == b:
                       split[i] = new_token
                       new_split = list(filter(lambda x: x not in [b], split))
                       self.splits[word] = new_split
        return self.splits

    def tokenize(self, text):
        """
        Tokenize a given text using the trained BPE tokenizer.

        Args:
            text (str): The text to be tokenized.

        Returns:
            list[str]: A list of tokens obtained after applying BPE tokenization.
        """

        pre_tokenized_text = text.split()
        splits_text = [[l for l in word] for word in pre_tokenized_text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits_text):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits_text[idx] = split
        result = sum(splits_text, [])
        return result

# Feature Extraction

## One-Hot Encoding Vectorization

In [84]:
def one_hot_vectorize(sentences, type2index):
    """
    One-hot encode a list of sentences.

    param:
        list of list of tokens e.g. [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence'], ...]
        type2index: dictionary mapping words to their index in the vocabulary e.g. {'word1': 0, 'word2': 1, 'word3': 2, ...}
    return:
        one_hot_sentences: 2d numpy array of one-hot encoded sentences e.g. [[1, 0, 0, 1, ...], [0, 1, 1, 0, ...], ...]
    """
    one_hot_encoding = np.array(np.zeros((len(type2index.items()), len(type2index.items()))))
    for sentence in sentences:
        for word in sentence:
            word_index = type2index[word]
            one_hot_encoding[word_index, word_index] = 1
    return one_hot_encoding

def simple_binary_vectorization(sentences, type2index):
    """
    Binary text-vectorization of a list of sentences.

    param:
        list of list of tokens e.g. [['This', 'is', 'a', 'sentence'], ['This', 'is', 'another', 'sentence'], ...]
        type2index: dictionary mapping words to their index in the vocabulary e.g. {'word1': 0, 'word2': 1, 'word3': 2, ...}
    return:
        one_hot_sentences: 2d numpy array of one-hot encoded sentences e.g. [[1, 0, 0, 1, ...], [0, 1, 1, 0, ...], ...]
    """
    vectors = []
    for sentence in sentences:
        vector = np.zeros(len(type2index.items()))
        for word in sentence:
             word_index = type2index[word]
             vector[word_index] = 1
        vectors.append(np.array(vector))
    return np.array(vectors)

def bow_vectorization(sentences, type2index):
    vectors = []
    for sentence in sentences:
        vector = np.zeros(len(type2index.items()))
        for word in sentence:
             word_index = type2index[word]
             vector[word_index] += 1
        vectors.append(np.array(vector))
    return np.array(vectors)
     

## Word-Based Tokenization Features


In [85]:
# Store training data text as list of tweets
train_corpus = train_df['text'].tolist()
tokenized_train_corpus = whitespace_tokenize(train_corpus)
num_tokens = count_tokens(tokenized_train_corpus)
print('Number of tokens in corpus: ', num_tokens)
type2count = create_type_counts(tokenized_train_corpus)
# Sort types by counts
type2count = dict(sorted(type2count.items(), key=lambda x: x[1], reverse=True))
index2type, type2index = create_vocabulary(type2count, min_count=1)
# It's good practice to add a special token for unknown words and padding (to make all sentences in training batches the same length)
type2index['<UNK>'] = len(index2type)
index2type.append('<UNK>')
type2index['<PAD>'] = len(index2type)
index2type.append('<PAD>')
x_train_word_based_feature_matrix = bow_vectorization(tokenized_train_corpus, type2index)

dev_corpus = dev_df['text'].tolist()
tokenized_dev_corpus = whitespace_tokenize(dev_corpus)
num_tokens = count_tokens(tokenized_dev_corpus)
print('Number of tokens in corpus: ', num_tokens)
type2count_dev = create_type_counts(tokenized_dev_corpus)
type2count_dev = dict(sorted(type2count_dev.items(), key=lambda x: x[1], reverse=True))
index2type_dev, type2index_dev = create_vocabulary(type2count_dev, min_count=1)
x_dev_word_based_feature_matrix = bow_vectorization(tokenized_dev_corpus, type2index_dev)


test_corpus = test_df['text'].tolist()
tokenized_test_corpus = whitespace_tokenize(test_corpus)
num_tokens = count_tokens(tokenized_test_corpus)
print('Number of tokens in corpus: ', num_tokens)
type2count_test = create_type_counts(tokenized_test_corpus)
type2count_test = dict(sorted(type2count_test.items(), key=lambda x: x[1], reverse=True))
index2type_test, type2index_test = create_vocabulary(type2count_test, min_count=1)
x_test_word_based_feature_matrix = bow_vectorization(tokenized_test_corpus, type2index_test)


Number of tokens in corpus:  13122
Number of tokens in corpus:  3064
Number of tokens in corpus:  5401


## BPE Tokenization Features

In [79]:
# Train BPE
bpe = BPETokenizer(tokenized_train_corpus, vocab_size=1000)
merges = bpe.train()
#print("Merges", merges)
# Apply to our dataset
train_df['bpe_text'] = train_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))
dev_df['bpe_text'] = dev_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))
test_df['bpe_text'] = test_df['text'].apply(lambda x: ' '.join(bpe.tokenize(x)))
train_df.head()

bpe_train_corpus = train_df['bpe_text'].tolist()
tokenized_bpe_train_corpus = whitespace_tokenize(bpe_train_corpus)
bpe_type2count = create_type_counts(tokenized_bpe_train_corpus)
# Sort types by counts
bpe_type2count = dict(sorted(bpe_type2count.items(), key=lambda x: x[1], reverse=True))
bpe_index2type, bpe_type2index = create_vocabulary(bpe_type2count, min_count=1)
x_train_bpe_feature_matrix = simple_binary_vectorization(tokenized_bpe_train_corpus, bpe_type2index)


# Model Training


## Compute Probabilities


In [47]:
class NaiveBayesClassifier():
    
    def __init__(self, x_train, y_train, type2index):
        self.class_priors = {}
        self.likelihoods = {}
        self.word_counts = type2index
        self.vocabulary = set()
        self.y = y_train
        self.x = x_train
    def train(self):
        # Calculate class prior probabilities
        print("Training Naive Bayes Classifier")
        self.prior_probs()
        print(self.class_priors)
        #for label in set(y_train)          
    
    def prior_probs(self):   
        positive_sentences = 0
        for label in self.y:
            if label == "positive":
                positive_sentences += 1
        self.class_priors["positive"] = positive_sentences/len(self.y)
        self.class_priors["negative"] = 1 - self.class_priors["positive"]
    def likelihood(self):
        print("test")
    def predict(self):
        print("Predicting Naive Bayes")
    
        

## Compute Likelihood

In [88]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


#x_train_word_based_feature_matrix = jnp.array(x_train_word_based_feature_matrix, dtype=jnp.float16)
#x_train_bpe_feature_matrix = jnp.array(x_train_bpe_feature_matrix, dtype=jnp.float16)#
y_train = train_df["label"]
y_train = jnp.array(y_train.map({"positive": 1, "negative": 0}), dtype=jnp.float16)

x_train_word_based_feature_matrix


#nb = NaiveBayesClassifier(labels,type2index)

#nb = MultinomialNB()
#nb.fit(x_train_word_based_feature_matrix, y_train)
#y_pred = nb.predict(x_dev_word_based_feature_matrix)




array([[0., 1., 0., ..., 0., 0., 0.],
       [3., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])